### 1. Carrega o arquivo com as sentenças, features e classificação

In [1]:
import pandas as pd

data_frame = pd.read_csv('dataset_sentencas_processadas_400.csv')

# sentenca_original = sentenças obtidas do trabalho de Gabriela
# sentenca_processada1 = remoção das anotações feitas por Gabriela
# sentenca_processada2 = sem pontuações
# sentenca_processada3_1 = sem acentos
# sentenca_processada3_2 = sem stopwords
# sentenca_processada3_3 = letras minúsculas
# sentenca_processada3_4 = apenas as raizes das palavras (stemmer)

In [2]:
data_frame.head(1)

,classificacao,sentenca,sentenca_processada1,sentenca_processada2,sentenca_processada3_1,sentenca_processada3_2,sentenca_processada3_3,sentenca_processada3_4
0,1.0,Parece estar no Reino da Dinamarca.,Parece estar no Reino da Dinamarca.,Parece estar no Reino da Dinamarca,Parece estar no Reino da Dinamarca,Parece estar Reino Dinamarca,parece estar reino dinamarca,parec est rein dinamarc


In [3]:
print(data_frame['classificacao'].value_counts())

0.0    200
1.0    200
Name: classificacao, dtype: int64


### 4. BOW (Bag of Words)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vetorizar = CountVectorizer(lowercase = False) #, max_features=10000);

bag_of_words = vetorizar.fit_transform(data_frame['sentenca_processada3_4'].values.astype('U'))

vocabulario = vetorizar.get_feature_names()

In [5]:
print(bag_of_words.shape)

(400, 1389)


In [6]:
# print(bag_of_words)

#### 4.1. Utiliza conjunto de treino/teste (90/10%) para cálculo Precisão, Acurácia, F1-Score e Recall

In [7]:
from sklearn.model_selection import train_test_split
from sklearn import tree
import numpy as np
from sklearn import metrics

max_f1 = 0
max_precision = 0
max_i_f1 = 0
max_i_precision = 0

for i in range(100):
    # Separa o corpus em conjunto de dados de treino e de teste.
    treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words, data_frame['classificacao'], random_state = i, test_size = 0.1)

    # Treina o modelo usando o conjunto de dados de treino:
    classificador = tree.DecisionTreeClassifier()
    classificador.fit(treino, classe_treino)

    # realiza a classificação usando os dados de teste e o modelo treinado anteriormente:
    previsao = classificador.predict_proba(teste)

    # transforma as saídas classificadas de acordo com um limiar:
    previsao_bool = previsao[:,1] >= 0.5

    # transforma as saídas classificadas (booleanas) em valores inteiros:
    previsao_int = previsao_bool.astype(np.int)
    
    f1_score = metrics.f1_score(classe_teste, previsao_int)
    accuracy = metrics.accuracy_score(classe_teste, previsao_int)
    precision = metrics.precision_score(classe_teste, previsao_int)
    recall = metrics.recall_score(classe_teste, previsao_int)
    
    if(max_f1 < f1_score): 
        max_f1 = f1_score
        max_i_f1 = i
        
    if(max_precision < precision): 
        max_precision = precision
        max_i_precision = i

    # Apresenta os resultados de avaliação do algoritmo de classificação
    print(i, '\t', round(accuracy,2), '\t', round(recall,2), '\t', round(precision,2), '\t', round(f1_score,2))
    
print(max_i_precision, round(max_precision, 2), max_i_f1, round(max_f1, 2))

0 	 0.5 	 0.29 	 0.7 	 0.41
1 	 0.57 	 0.42 	 0.57 	 0.48
2 	 0.45 	 0.23 	 0.5 	 0.31
3 	 0.5 	 0.5 	 0.55 	 0.52
4 	 0.45 	 0.29 	 0.58 	 0.39
5 	 0.52 	 0.22 	 0.44 	 0.3
6 	 0.42 	 0.3 	 0.5 	 0.38
7 	 0.5 	 0.26 	 0.45 	 0.33
8 	 0.55 	 0.32 	 0.55 	 0.4
9 	 0.42 	 0.27 	 0.46 	 0.34
10 	 0.48 	 0.33 	 0.5 	 0.4
11 	 0.62 	 0.38 	 0.55 	 0.44
12 	 0.48 	 0.25 	 0.31 	 0.28
13 	 0.52 	 0.4 	 0.53 	 0.46
14 	 0.52 	 0.35 	 0.67 	 0.46
15 	 0.5 	 0.47 	 0.42 	 0.44
16 	 0.55 	 0.42 	 0.53 	 0.47
17 	 0.52 	 0.29 	 0.6 	 0.39
18 	 0.55 	 0.4 	 0.57 	 0.47
19 	 0.52 	 0.4 	 0.53 	 0.46
20 	 0.38 	 0.38 	 0.47 	 0.42
21 	 0.5 	 0.32 	 0.73 	 0.44
22 	 0.38 	 0.12 	 0.17 	 0.14
23 	 0.62 	 0.45 	 0.69 	 0.55
24 	 0.48 	 0.32 	 0.67 	 0.43
25 	 0.72 	 0.67 	 0.62 	 0.65
26 	 0.5 	 0.42 	 0.47 	 0.44
27 	 0.6 	 0.48 	 0.67 	 0.56
28 	 0.45 	 0.21 	 0.36 	 0.27
29 	 0.62 	 0.37 	 0.7 	 0.48
30 	 0.38 	 0.29 	 0.47 	 0.36
31 	 0.62 	 0.56 	 0.53 	 0.55
32 	 0.57 	 0.58 	 0.55 	 0.56
33 	 0.6

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn import metrics

max_f1 = 0
max_precision = 0
max_i_f1 = 0
max_i_precision = 0

for i in range(100):
    # Separa o corpus em conjunto de dados de treino e de teste.
    treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words, data_frame['classificacao'], random_state = i, test_size = 0.1)

    # Treina o modelo usando o conjunto de dados de treino:
    classificador = LogisticRegression()
    classificador.fit(treino, classe_treino)

    # realiza a classificação usando os dados de teste e o modelo treinado anteriormente:
    previsao = classificador.predict_proba(teste)

    # transforma as saídas classificadas de acordo com um limiar:
    previsao_bool = previsao[:,1] >= 0.5

    # transforma as saídas classificadas (booleanas) em valores inteiros:
    previsao_int = previsao_bool.astype(np.int)
    
    f1_score = metrics.f1_score(classe_teste, previsao_int)
    accuracy = metrics.accuracy_score(classe_teste, previsao_int)
    precision = metrics.precision_score(classe_teste, previsao_int)
    recall = metrics.recall_score(classe_teste, previsao_int)
    
    if(max_f1 < f1_score): 
        max_f1 = f1_score
        max_i_f1 = i
        
    if(max_precision < precision): 
        max_precision = precision
        max_i_precision = i

    # Apresenta os resultados de avaliação do algoritmo de classificação
    print(i, '\t', round(accuracy,2), '\t', round(recall,2), '\t', round(precision,2), '\t', round(f1_score,2))
    
print(max_i_precision, round(max_precision, 2), max_i_f1, round(max_f1, 2))

0 	 0.6 	 0.58 	 0.7 	 0.64
1 	 0.52 	 0.32 	 0.5 	 0.39
2 	 0.6 	 0.5 	 0.69 	 0.58
3 	 0.57 	 0.5 	 0.65 	 0.56
4 	 0.48 	 0.46 	 0.58 	 0.51
5 	 0.45 	 0.33 	 0.38 	 0.35
6 	 0.45 	 0.52 	 0.52 	 0.52
7 	 0.55 	 0.42 	 0.53 	 0.47
8 	 0.55 	 0.42 	 0.53 	 0.47
9 	 0.65 	 0.73 	 0.67 	 0.7
10 	 0.5 	 0.43 	 0.53 	 0.47
11 	 0.6 	 0.56 	 0.5 	 0.53
12 	 0.5 	 0.56 	 0.41 	 0.47
13 	 0.5 	 0.5 	 0.5 	 0.5
14 	 0.65 	 0.52 	 0.8 	 0.63
15 	 0.52 	 0.71 	 0.46 	 0.56
16 	 0.57 	 0.68 	 0.54 	 0.6
17 	 0.62 	 0.57 	 0.67 	 0.62
18 	 0.68 	 0.55 	 0.73 	 0.63
19 	 0.7 	 0.6 	 0.75 	 0.67
20 	 0.57 	 0.58 	 0.67 	 0.62
21 	 0.5 	 0.44 	 0.65 	 0.52
22 	 0.45 	 0.35 	 0.35 	 0.35
23 	 0.5 	 0.45 	 0.5 	 0.47
24 	 0.62 	 0.64 	 0.73 	 0.68
25 	 0.57 	 0.73 	 0.46 	 0.56
26 	 0.52 	 0.47 	 0.5 	 0.49
27 	 0.62 	 0.57 	 0.67 	 0.62
28 	 0.45 	 0.42 	 0.42 	 0.42
29 	 0.5 	 0.42 	 0.47 	 0.44
30 	 0.52 	 0.54 	 0.62 	 0.58
31 	 0.7 	 0.88 	 0.58 	 0.7
32 	 0.6 	 0.63 	 0.57 	 0.6
33 	 0.57 	 0.4

In [9]:
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
from sklearn import metrics

max_f1 = 0
max_precision = 0
max_i_f1 = 0
max_i_precision = 0

for i in range(100):
    # Separa o corpus em conjunto de dados de treino e de teste.
    treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words, data_frame['classificacao'], random_state = i, test_size = 0.1)

    # Treina o modelo usando o conjunto de dados de treino:
    classificador = svm.SVC(gamma='auto', C=1.0, kernel='linear', probability=True)
    classificador.fit(treino, classe_treino)

    # realiza a classificação usando os dados de teste e o modelo treinado anteriormente:
    previsao = classificador.predict_proba(teste)

    # transforma as saídas classificadas de acordo com um limiar:
    previsao_bool = previsao[:,1] >= 0.5

    # transforma as saídas classificadas (booleanas) em valores inteiros:
    previsao_int = previsao_bool.astype(np.int)
    
    f1_score = metrics.f1_score(classe_teste, previsao_int)
    accuracy = metrics.accuracy_score(classe_teste, previsao_int)
    precision = metrics.precision_score(classe_teste, previsao_int)
    recall = metrics.recall_score(classe_teste, previsao_int)
    
    if(max_f1 < f1_score): 
        max_f1 = f1_score
        max_i_f1 = i
        
    if(max_precision < precision): 
        max_precision = precision
        max_i_precision = i

    # Apresenta os resultados de avaliação do algoritmo de classificação
    print(i, '\t', round(accuracy,2), '\t', round(recall,2), '\t', round(precision,2), '\t', round(f1_score,2))
    
print(max_i_precision, round(max_precision, 2), max_i_f1, round(max_f1, 2))

0 	 0.57 	 0.33 	 0.89 	 0.48
1 	 0.45 	 0.84 	 0.46 	 0.59
2 	 0.6 	 0.5 	 0.69 	 0.58
3 	 0.62 	 0.5 	 0.73 	 0.59
4 	 0.38 	 0.38 	 0.47 	 0.42
5 	 0.42 	 0.44 	 0.38 	 0.41
6 	 0.45 	 0.52 	 0.52 	 0.52
7 	 0.57 	 0.58 	 0.55 	 0.56
8 	 0.52 	 0.63 	 0.5 	 0.56
9 	 0.52 	 0.55 	 0.57 	 0.56
10 	 0.52 	 0.67 	 0.54 	 0.6
11 	 0.48 	 0.75 	 0.41 	 0.53
12 	 0.35 	 0.5 	 0.31 	 0.38
13 	 0.4 	 0.5 	 0.42 	 0.45
14 	 0.55 	 0.43 	 0.67 	 0.53
15 	 0.55 	 0.88 	 0.48 	 0.62
16 	 0.5 	 0.68 	 0.48 	 0.57
17 	 0.55 	 0.62 	 0.57 	 0.59
18 	 0.48 	 0.65 	 0.48 	 0.55
19 	 0.4 	 0.65 	 0.43 	 0.52
20 	 0.55 	 0.54 	 0.65 	 0.59
21 	 0.52 	 0.44 	 0.69 	 0.54
22 	 0.4 	 0.65 	 0.38 	 0.48
23 	 0.45 	 0.45 	 0.45 	 0.45
24 	 0.5 	 0.44 	 0.65 	 0.52
25 	 0.55 	 1.0 	 0.45 	 0.62
26 	 0.62 	 0.74 	 0.58 	 0.65
27 	 0.55 	 0.52 	 0.58 	 0.55
28 	 0.35 	 0.37 	 0.33 	 0.35
29 	 0.48 	 0.53 	 0.45 	 0.49
30 	 0.48 	 0.46 	 0.58 	 0.51
31 	 0.5 	 1.0 	 0.44 	 0.62
32 	 0.55 	 0.74 	 0.52 	 0.61
33

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn import metrics

max_f1 = 0
max_precision = 0
max_i_f1 = 0
max_i_precision = 0

for i in range(100):
    # Separa o corpus em conjunto de dados de treino e de teste.
    treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words, data_frame['classificacao'], random_state = i, test_size = 0.1)

    max_f1_j = 0
    max_precision_j = 0
    max_j_f1 = 0
    max_j_precision = 0

    for j in range(50):
        # Treina o modelo usando o conjunto de dados de treino:
        classificador = MLPClassifier(activation='relu', solver='adam', max_iter=200, alpha=0.001, random_state=j)
        classificador.fit(treino, classe_treino)

        # realiza a classificação usando os dados de teste e o modelo treinado anteriormente:
        previsao = classificador.predict_proba(teste)

        # transforma as saídas classificadas de acordo com um limiar:
        previsao_bool = previsao[:,1] >= 0.5

        # transforma as saídas classificadas (booleanas) em valores inteiros:
        previsao_int = previsao_bool.astype(np.int)

        f1_score = metrics.f1_score(classe_teste, previsao_int)
        accuracy = metrics.accuracy_score(classe_teste, previsao_int)
        precision = metrics.precision_score(classe_teste, previsao_int)
        recall = metrics.recall_score(classe_teste, previsao_int)

        if(max_f1_j < f1_score): 
            max_f1_j = f1_score
            max_j_f1 = j

        if(max_precision_j < precision): 
            max_precision_j = precision
            max_j_precision = j

        # Apresenta os resultados de avaliação do algoritmo de classificação
        print(i, '\t', j, '\t', round(accuracy,2), '\t', round(recall,2), '\t', round(precision,2), '\t', round(f1_score,2))

    print(max_j_precision, round(max_precision_j, 2), max_j_f1, round(max_f1_j, 2))
    
    if(max_f1 < max_f1_j): 
        max_f1 = max_f1_j
        max_i_f1 = i

    if(max_precision < max_precision_j): 
        max_precision = max_precision_j
        max_i_precision = i
    
print(max_i_precision, round(max_precision, 2), max_i_f1, round(max_f1, 2))

/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0 	 0 	 0.48 	 0.46 	 0.58 	 0.51
0 	 1 	 0.5 	 0.5 	 0.6 	 0.55
0 	 2 	 0.5 	 0.5 	 0.6 	 0.55
0 	 3 	 0.5 	 0.5 	 0.6 	 0.55
0 	 4 	 0.52 	 0.5 	 0.63 	 0.56
0 	 5 	 0.5 	 0.5 	 0.6 	 0.55
0 	 6 	 0.52 	 0.5 	 0.63 	 0.56
0 	 7 	 0.5 	 0.5 	 0.6 	 0.55
0 	 8 	 0.52 	 0.54 	 0.62 	 0.58
0 	 9 	 0.5 	 0.5 	 0.6 	 0.55
0 	 10 	 0.52 	 0.5 	 0.63 	 0.56
0 	 11 	 0.5 	 0.5 	 0.6 	 0.55
0 	 12 	 0.52 	 0.5 	 0.63 	 0.56
0 	 13 	 0.52 	 0.5 	 0.63 	 0.56
0 	 14 	 0.52 	 0.5 	 0.63 	 0.56
0 	 15 	 0.48 	 0.46 	 0.58 	 0.51
0 	 16 	 0.5 	 0.5 	 0.6 	 0.55
0 	 17 	 0.52 	 0.5 	 0.63 	 0.56
0 	 18 	 0.5 	 0.5 	 0.6 	 0.55
0 	 19 	 0.5 	 0.5 	 0.6 	 0.55
0 	 20 	 0.5 	 0.5 	 0.6 	 0.55
0 	 21 	 0.52 	 0.5 	 0.63 	 0.56
0 	 22 	 0.52 	 0.5 	 0.63 	 0.56
0 	 23 	 0.5 	 0.5 	 0.6 	 0.55
0 	 24 	 0.5 	 0.46 	 0.61 	 0.52
0 	 25 	 0.52 	 0.5 	 0.63 	 0.56
0 	 26 	 0.5 	 0.5 	 0.6 	 0.55
0 	 27 	 0.5 	 0.5 	 0.6 	 0.55
0 	 28 	 0.5 	 0.5 	 0.6 	 0.55
0 	 29 	 0.5 	 0.5 	 0.6 	 0.55
0 	 30 	 0.5 	 0.46

/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


3 	 16 	 0.6 	 0.5 	 0.69 	 0.58
3 	 17 	 0.6 	 0.5 	 0.69 	 0.58
3 	 18 	 0.62 	 0.5 	 0.73 	 0.59
3 	 19 	 0.62 	 0.45 	 0.77 	 0.57
3 	 20 	 0.62 	 0.5 	 0.73 	 0.59
3 	 21 	 0.6 	 0.5 	 0.69 	 0.58
3 	 22 	 0.62 	 0.5 	 0.73 	 0.59
3 	 23 	 0.62 	 0.5 	 0.73 	 0.59
3 	 24 	 0.65 	 0.5 	 0.79 	 0.61
3 	 25 	 0.6 	 0.5 	 0.69 	 0.58
3 	 26 	 0.62 	 0.5 	 0.73 	 0.59
3 	 27 	 0.62 	 0.5 	 0.73 	 0.59
3 	 28 	 0.6 	 0.45 	 0.71 	 0.56
3 	 29 	 0.65 	 0.5 	 0.79 	 0.61
3 	 30 	 0.65 	 0.5 	 0.79 	 0.61
3 	 31 	 0.65 	 0.5 	 0.79 	 0.61
3 	 32 	 0.65 	 0.5 	 0.79 	 0.61
3 	 33 	 0.62 	 0.5 	 0.73 	 0.59
3 	 34 	 0.62 	 0.5 	 0.73 	 0.59
3 	 35 	 0.62 	 0.5 	 0.73 	 0.59
3 	 36 	 0.62 	 0.5 	 0.73 	 0.59
3 	 37 	 0.65 	 0.5 	 0.79 	 0.61
3 	 38 	 0.65 	 0.5 	 0.79 	 0.61
3 	 39 	 0.62 	 0.5 	 0.73 	 0.59
3 	 40 	 0.65 	 0.5 	 0.79 	 0.61
3 	 41 	 0.65 	 0.5 	 0.79 	 0.61
3 	 42 	 0.65 	 0.5 	 0.79 	 0.61
3 	 43 	 0.62 	 0.5 	 0.73 	 0.59
3 	 44 	 0.6 	 0.5 	 0.69 	 0.58
3 	 45 	 0.65 	 0.

8 	 3 	 0.5 	 0.37 	 0.47 	 0.41
8 	 4 	 0.5 	 0.37 	 0.47 	 0.41
8 	 5 	 0.5 	 0.37 	 0.47 	 0.41
8 	 6 	 0.5 	 0.37 	 0.47 	 0.41
8 	 7 	 0.5 	 0.37 	 0.47 	 0.41
8 	 8 	 0.5 	 0.37 	 0.47 	 0.41
8 	 9 	 0.5 	 0.37 	 0.47 	 0.41
8 	 10 	 0.5 	 0.37 	 0.47 	 0.41
8 	 11 	 0.5 	 0.37 	 0.47 	 0.41
8 	 12 	 0.5 	 0.37 	 0.47 	 0.41
8 	 13 	 0.5 	 0.37 	 0.47 	 0.41
8 	 14 	 0.5 	 0.37 	 0.47 	 0.41
8 	 15 	 0.5 	 0.37 	 0.47 	 0.41
8 	 16 	 0.5 	 0.37 	 0.47 	 0.41
8 	 17 	 0.5 	 0.37 	 0.47 	 0.41
8 	 18 	 0.5 	 0.37 	 0.47 	 0.41
8 	 19 	 0.5 	 0.37 	 0.47 	 0.41
8 	 20 	 0.5 	 0.37 	 0.47 	 0.41
8 	 21 	 0.5 	 0.37 	 0.47 	 0.41
8 	 22 	 0.5 	 0.37 	 0.47 	 0.41
8 	 23 	 0.5 	 0.37 	 0.47 	 0.41
8 	 24 	 0.52 	 0.42 	 0.5 	 0.46
8 	 25 	 0.5 	 0.37 	 0.47 	 0.41
8 	 26 	 0.48 	 0.37 	 0.44 	 0.4
8 	 27 	 0.5 	 0.37 	 0.47 	 0.41
8 	 28 	 0.5 	 0.37 	 0.47 	 0.41
8 	 29 	 0.5 	 0.37 	 0.47 	 0.41
8 	 30 	 0.5 	 0.37 	 0.47 	 0.41
8 	 31 	 0.5 	 0.37 	 0.47 	 0.41
8 	 32 	 0.5 	 0.37 	